In [3]:
# this is a simple script to load the OpenAI API key from a .env file
# make sure to create a .env file with your OpenAI API key
# also .env file is added to .gitignore so it won't be pushed to the git repository
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')
print("OpenAI API Key loaded successfully.")
# Get the stock API key
stock_api_key = os.getenv('ALPHA_VANTAGE_API_KEY')
print("Stock API Key loaded successfully.")

OpenAI API Key loaded successfully.
Stock API Key loaded successfully.


In [10]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import math
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
import requests

# define a calculator function and tool
def calculator(expression: str):
    """Evaluates a math expression and returns the result."""
    return eval(expression)

calculator_tool = Tool(
    name="Calculator",
    func=calculator,
    description="A simple calculator that can evaluate math expressions."
)

# Create a Wikipedia search tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Create a stock query *custom* tool
def get_stock_price(symbol: str):
    """Fetches the latest stock price for a given ticker symbol."""
    api_key = stock_api_key  # Replace with your API key
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}"    
    response = requests.get(url)
    data = response.json()    
    if "Global Quote" in data:
        price = data["Global Quote"].get("05. price", "N/A")
        return f"The current price of {symbol.upper()} is ${price}."
    else:
        return "Invalid stock symbol or API limit reached."

# Define the stock tool
stock_tool = Tool(
    name="Stock Price Checker",
    func=get_stock_price,
    description="Fetches the latest stock price for a given ticker symbol (e.g., AAPL, TSLA, MSFT)."
)




# Define the LLM model
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Extend the agent with both tools
agent = initialize_agent(
    tools=[calculator_tool, wikipedia, stock_tool],  
    llm=llm,  
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  
    verbose=True  
)

## use the tools
# Ask a question that requires Wikipedia search
response = agent.invoke("Who was Albert Einstein?")
print(response)

response = agent.invoke("What is 34*33")
print(response)

response = agent.invoke("What is stock price for LULU ?")
print(response)



> Entering new AgentExecutor chain...
This is a general knowledge question about a historical figure. I can use Wikipedia to find information about Albert Einstein.
Action: wikipedia
Action Input: "Albert Einstein"
Observation: Page: Albert Einstein
Summary: Albert Einstein (, EYEN-styne; German: [ˈalbɛʁt ˈʔaɪnʃtaɪn] ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is best known for developing the theory of relativity. Einstein also made important contributions to quantum mechanics. His mass–energy equivalence formula E = mc2, which arises from special relativity, has been called "the world's most famous equation". He received the 1921 Nobel Prize in Physics for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect.
Born in the German Empire, Einstein moved to Switzerland in 1895, forsaking his German citizenship (as a subject of the Kingdom of Württemberg) the following year. In 1897, at the age of se